In [1]:
import os
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt

/tmp/ipykernel_15531/1673349187.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [ ]:
data = pd.DataFrame({
    'wind_speed': np.random.rand(1000),
    'wind_direction': np.random.rand(1000),
    'time_info': pd.date_range(start='2022-01-01', periods=1000, freq='H'),
    'generation': np.random.rand(1000)
})

# 데이터 전처리
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['wind_speed', 'wind_direction', 'generation']])

# 시퀀스 데이터 생성
sequence_length = 48
X = []
y = []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i+sequence_length, :-1])
    y.append(data_scaled[i+sequence_length, -1])
X = np.array(X)
y = np.array(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(units=50, activation='relu', 
               input_shape=(sequence_length, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=32)

# 테스트 데이터로 예측 수행
predicted_scaled = model.predict(X_test)
predicted = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], predicted_scaled), axis=1))[:, -1]

# 실제 발전량과 예측 발전량 비교
actual_generation = scaler.inverse_transform(y_test.reshape(-1, 1))
print(actual_generation)
print(predicted)